ideas

if edit1 word is less common that edit2 word, then choose edit2 


In [32]:
from Levenshtein import distance as levenshtein_distance_function
levenshtein_distance("urytop", "ahjbfds")

7

In [112]:
from enum import Enum
import numpy as np

class EditType(Enum):
    none = 0
    insert = 1
    replace = 2
    delete = 3
    transpose = 4
    
    def __lt__(self, other):
        self.value < other.value
        
    def probability(self):
        """
            A paper by Kukich (1992) suggested the following
            error rates based on an analysis of spelling errors
            in various databases:

            Substitution errors: 80%
            Deletion errors: 10%
            Insertion errors: 5%
            Transposition errors: 5%
        """
        
        
        if self == EditType.none:
            return 0
        elif self == EditType.insert:
            return 0.05
        elif self == EditType.replace:
            return 0.8
        elif self == EditType.delete:
            return 0.1
        else:
            return 0.05

class Edit:
    
    def __init__(self, 
                 word, 
                 edit,
                 edit_type: EditType,
                 edit_word_probability,
                 keyboard_distance,
                 prev_edit = None):
        self.word = word # the misspelled word
        self.edit = edit # the edit that would fix the misspelled word
        self.edit_type = edit_type # edit type, refer to the enum above
        self.prev_edit = prev_edit # reference to the previous edit, if any
        self.edit_word_probability = edit_word_probability # the probability of the suggested edit to occur in the whole text curpus
        self.keyboard_distance = keyboard_distance # for example, if edit_type is replace, keyboard_distance is the manhattan distance between the correct and wrong letters on a qwerty layout keyabord
        self.levenshtein_distance = levenshtein_distance_function(word, edit)
        
    def __hash__(self):
        return hash((self.edit, self.edit_type))

    def __eq__(self,other):
        return self.edit == other.edit and self.edit_type == other.edit_type
    
    def __repr__(self):
        return f"""
        Edit of
            content = {self.word}
            edit = {self.edit}
            type = {self.edit_type.name}
            edit word probaility = {self.edit_word_probability}
            keyboard_distance = {self.keyboard_distance}
            levenshtein distance = {self.levenshtein_distance}
            previous edit = 
                {self.prev_edit}
        """
    
    def probability(self, keyboard_weight = 1.0, edit_type_weight = 1.0, word_probability_weight = 5.0):
        keyboard_weight = keyboard_weight
        edit_type_weight = edit_type_weight
        word_probability_weight = word_probability_weight  # Give more weight to word probability

        keyboard_distance_score = 1 / (self.keyboard_distance + 1e-3)  # Add small constant to avoid division by zero
        edit_type_probability = self.edit_type.probability()

        # Use logarithm of word probability to provide a stronger differentiation between words
        word_probability = np.log(self.edit_word_probability + 1e-10)  # Add small constant to ensure the logarithm is defined

        return (
            keyboard_weight * keyboard_distance_score +
            edit_type_weight * edit_type_probability +
            word_probability_weight * word_probability
        )



In [103]:
import re
from collections import Counter
from constants import keyboard_letters, neighbour_letters, keyboard_layouts

class SpellChecker:
    
    def __init__(self, language = "en"):
        
        file_to_open = f'big_{language}.txt'
        
        with open(file_to_open, "r") as file:
            raw_text = file.read()
            raw_words = re.findall(r'\w+', raw_text.lower())
            self.WORDS = Counter(raw_words)
        
        self.N = sum(self.WORDS.values())
        self.letters = keyboard_letters[language]
        self.neighbour_letters = neighbour_letters[language]
        self.keyboard_layout = keyboard_layouts[language]
        
    def keyboard_distance(self, key1, key2):

        key_coordinates = {}

        for i, row in enumerate(self.keyboard_layout):
            for j, key in enumerate(row):
                key_coordinates[key] = (i, j)

        if key1 not in key_coordinates or key2 not in key_coordinates:
            raise ValueError("Both keys must be on the keyboard")

        x1, y1 = key_coordinates[key1]
        x2, y2 = key_coordinates[key2]

        return abs(x1 - x2) + abs(y1 - y2)
        
    def P(self, word): 
        "Probability of `word`."
        
        # first sort by probability, then sort by edit_type, then sort by keyboard distance, if any
        return self.WORDS[word] / self.N

    def correction(self, word): 
        "Most probable spelling correction for word."
        
        # correction is defined as following:
        candidates = list(self.candidates(word))
        
        candidates.sort(key = lambda x: checker.P(x), reverse = True)
        candidates.sort(key = lambda x: x.keyboard_distance)
        candidates.sort(key = lambda x: x.edit_type.value)
        
        return candidates[0]

    def candidates(self, word): 
        "Generate possible spelling corrections for word."
        return (self.known([Edit(word, word, EditType.none, 0, 0)]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [Edit(word, word, EditType.none, 0, 0)])

    def known(self, edits): 
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in edits if w.edit in self.WORDS)

    def edits1(self, word, prev_edit: Edit = None):
        "All edits that are one edit away from `word`."
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        
        deletes    = set(self.__edits_deletes(word, splits, prev_edit))
        transposes = set(self.__edits_transposes(word, splits, prev_edit))
        replaces   = set(self.__edits_replaces(word, splits, prev_edit))
        inserts    = set(self.__edits_inserts(word, splits, prev_edit))
        all_set = deletes.union(transposes).union(replaces).union(inserts)
        return all_set

    def edits2(self, word): 
        "All edits that are two edits away from `word`."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1.edit, prev_edit = e1))
    
    def __edits_transposes(self, word, splits, prev_edit: Edit):
        for L, R in splits:
            if len(R) > 1:
                edit_word = L + R[1] + R[0] + R[2:]
                yield Edit(
                    word = word, 
                    edit = edit_word,
                    edit_type = EditType.transpose,
                    prev_edit = prev_edit, 
                    edit_word_probability = self.P(edit_word),
                    keyboard_distance = 1
                )
    
    def __edits_inserts(self, word, splits, prev_edit: Edit):
        for L, R in splits:
            for c in self.letters:
                
                # Left's distance
                if len(L) > 0:
                    left = L[-1]
                    L_distance = self.keyboard_distance(left, c)
                else:
                    L_distance = -1
                
                edit_word = L + c + R
                yield Edit(
                    word = word,
                    edit = edit_word,
                    edit_type = EditType.insert,
                    prev_edit = prev_edit,
                    edit_word_probability = self.P(edit_word),
                    keyboard_distance = L_distance)
    
    def __edits_deletes(self, word, splits, prev_edit: Edit):
        for L, R in splits:
            if R:
                omitted = R[0]
                
                # Left's distance
                if len(L) > 0:
                    left = L[-1]
                    L_distance = self.keyboard_distance(left, omitted)
                else:
                    L_distance = -1
                    
                # Left's distance
                if len(R) > 0:
                    right = R[-1]
                    R_distance = self.keyboard_distance(right, omitted)
                else:
                    R_distance = -1
                
                key_distance = min(L_distance, R_distance)
                edit_word = L + R[1:]
                yield Edit(
                    word = word,
                    edit = edit_word,
                    edit_type = EditType.delete,
                    prev_edit = prev_edit, 
                    edit_word_probability = self.P(edit_word),
                    keyboard_distance = key_distance
                )
    
    def __edits_replaces(self, word, splits, prev_edit: Edit):
        for L, R in splits:
            if R:
                for c in self.letters:
                    
                    omitted = R[0]
                    key_distance = self.keyboard_distance(c, omitted)
                    edit_word = L + c + R[1:]
                    yield Edit(
                        word = word, 
                        edit = edit_word,
                        edit_type = EditType.replace, 
                        prev_edit = prev_edit,
                        edit_word_probability = self.P(edit_word),
                        keyboard_distance = key_distance
                    )
    

In [104]:
checker = SpellChecker()

In [105]:
len(checker.WORDS)

32198

In [106]:
checker.correction("receit")


        Edit of
            content = receit
            edit = receipt
            type = insert
            edit word probaility = 1.1653078877898144e-05
            keyboard_distance = 2
            levenshtein distance = 1
            previous edit = 
                None
        

In [107]:
candidates = list(checker.candidates("receit")) # receipt
candidates.sort(key = lambda x: checker.P(x), reverse = True)
candidates.sort(key = lambda x: x.keyboard_distance)
candidates.sort(key = lambda x: x.edit_type.value)
candidates

[
         Edit of
             content = receit
             edit = receipt
             type = insert
             edit word probaility = 1.1653078877898144e-05
             keyboard_distance = 2
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = receit
             edit = deceit
             type = replace
             edit word probaility = 3.585562731660967e-06
             keyboard_distance = 2
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = receit
             edit = recent
             type = replace
             edit word probaility = 4.750870619450781e-05
             keyboard_distance = 4
             levenshtein distance = 1
             previous edit = 
                 None
         ]

In [108]:
candidates = list(checker.candidates("receit")) # receipt
[(edit.probability(), edit.edit) for edit in candidates]

[(0.0, 'deceit'), (0.0, 'recent'), (0.0, 'receipt')]

In [109]:
checker.P(Edit("recent"))

TypeError: __init__() missing 4 required positional arguments: 'edit', 'edit_type', 'edit_word_probability', and 'keyboard_distance'

In [110]:
checker.correction("mre")


        Edit of
            content = mre
            edit = more
            type = insert
            edit word probaility = 0.001790092193781738
            keyboard_distance = 4
            levenshtein distance = 1
            previous edit = 
                None
        

In [111]:
candidates = checker.candidates("mre") # more
candidates

{
         Edit of
             content = mre
             edit = are
             type = replace
             edit word probaility = 0.0032538981789823275
             keyboard_distance = 7
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = mre
             edit = ere
             type = replace
             edit word probaility = 8.963906829152417e-07
             keyboard_distance = 6
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = mre
             edit = ire
             type = replace
             edit word probaility = 8.963906829152417e-07
             keyboard_distance = 3
             levenshtein distance = 1
             previous edit = 
                 None
         ,
 
         Edit of
             content = mre
             edit = mare
             type = insert
             edit word pro

In [87]:
checker.correction("evar")


        Edit
            content = ever
            type = replace
            previous edit = 
                None
            keyboard_distance = 3
        

In [35]:
candidates = checker.candidates("evar") # ever
candidates

{
         Edit
             content = ear
             type = delete
             previous edit = 
                 None
             keyboard_distance = 2
         ,
 
         Edit
             content = eva
             type = delete
             previous edit = 
                 None
             keyboard_distance = 0
         ,
 
         Edit
             content = ever
             type = replace
             previous edit = 
                 None
             keyboard_distance = 3
         }